# Get all file in directory
https://www.mkyong.com/python/python-how-to-list-all-files-in-a-directory/

In [39]:
import os

path = "/home/zeabus/Documents/supasan/2019_NLP/special_assignment/disease3_withtag"
list_file = []
for root , directory , files in os.walk( path ):
    for file_path in files:
        if '.xml' in file_path:
            list_file.append( os.path.join( root , file_path ) )

## Function Help print Thai Language

In [99]:
def print_member( list_data ):
    print( '[' )
    limit_range = len( list_data )
    for run in range(  0 , limit_range - 1):
        print( list_data[ run ] + ',')
    print( list_data[ limit_range - 1 ] + ']')

# Real xml files

In [207]:
# I will focuse on cl for NER and pos
# Input ise string don't split line for us
def parse_xml( data , default_class_type = 'O' ):
    word = ""
    part_of_speech = None
    can_except = False # True when file '\'
    tag_type = None
    tag_information = False
    class_type = default_class_type
    data_return = []
    run = 0
    while run < len(data):
        if data[ run ] == '\\' :
            can_except = True
            
        elif data[ run ] == "<" and (not can_except): # Mean is in tag
            if word != "": # clear old data
                word = clean_word( word )
                data_return.append( summary( word , part_of_speech , class_type ) )
                word = ""
                part_of_speech = None
                
            if tag_type != None: # Except is sign of tag close
                run += 1
                if data[ run ] == '/':
                    run += 1
                    while data[ run ] != '>' : # End tag
                        word += data[ run ]
                        run += 1
                    if tag_type == word :
                        tag_type = None
                        word = ""
                        part_of_speech = None
                        class_type = default_class_type
                    else:
                        print("Warning! failure parsing xml data don't same tag type")
                        break
                else:
                    print( "Warning! failure parsing xml data for complex tag")
                    break
            else: # Expect is start tag type
                run += 1
                tag_type = ""
                while data[ run ] not in [ '/' , ' ' , '>' ]:
                    tag_type += data[ run ]
                    run += 1
                if data[ run ] == ' ' : # good end
                    tag_information = True
                elif data[ run ] == '/' and data[ run + 1] == '>':
                    run += 1
                    tag_type = None
                elif data[ run ] == '>':
                    None
                else:
                    print( "Warning! failure parsing xml data for start tag type")
                    break
        elif tag_information : # Now process for get group
            if data[ run : run + 4 ] == 'cl=\"':
                run += 4
                class_type = ''
                while data[ run ] != '\"':
                    class_type += data[ run ]
                    run += 1
            elif data[ run ] == '>':
                tag_information = False
            else:
                None                
                        
        elif data[ run ] == '/' and (not can_except): # Mean turn POS
            if word != "": # Sure is POS
                part_of_speech = ""
                run += 1
                while data[ run ] not in ['<' , ' ']:
                    part_of_speech += data[ run ]
                    run += 1
                word = clean_word( word )
                data_return.append( summary( word , part_of_speech , class_type ) )
                word = ""
                part_of_speech = None
                if data[ run ] == '<':
                    run -= 1
                else:
                    None
            else:
                word += data[ run ]
                
        else: # Normally get data
            can_except = False
            word += data[ run ]       
        run += 1
    return data_return

def clean_word( data ):
    result = ""
    for character in data:
        if character not in " ":
            result += character
    return result

def result_parse_xml( datas ):
    for data in datas:
        print( "word : {} have pos is {} and class is {}".format( data['word'] , data['pos'] , data['cl']))
        
def summary( word , pos , cl ):
    return { 'word' : word , 'pos' : pos , 'cl' : cl }

In [218]:
target_file = list_file[8]
print( "target file is " + target_file )
data_file = open( target_file )

target file is /home/zeabus/Documents/supasan/2019_NLP/special_assignment/disease3_withtag/42.xml


In [229]:
data_line = data_file.readline()
print_member( data_line.split( ' ' ) )

[
นาย/ntit,
<NAME,
cl="PERSON",
case="false",
number="one">อนุทิน/npn,
_/blk,
ชาญวีรกูล/npn</NAME>,
_/blk,
<NAME,
cl="PERSON",
case="false",
number="one">รัฐมนตรีช่วยว่าการ/ncn,
กระทรวง/ncn,
สาธารณสุข/ncn</NAME>,
_/blk,
แถลง/vt,
ที่/prel,
จังหวัด/ncn,
<NAME,
cl="LOCATION">เชียงใหม่/npn</NAME>,
ว่า/conjncl,
<NAME,
cl="PERSON",
case="true",
number="many">ผู้/pref2,
ป่วย/vi</NAME>,
<NAME,
cl="DISEASE">ไข้หวัดนก/vi,
เกือบ/adv,
ร้อยทั้งร้อย/idm,
</NAME>,
มี/vt,
ประวัติ/ncn,
สัมผัส/vt,
<NAME,
cl="NON_HUMAN",
transmission="true">ไก่/ncn</NAME>,
,
_/blk,

]


In [230]:
answer = parse_xml( data_line )

In [231]:
result_parse_xml( answer )

word : นาย have pos is ntit and class is O
word : อนุทิน have pos is npn and class is PERSON
word : _ have pos is blk and class is PERSON
word : ชาญวีรกูล have pos is npn and class is PERSON
word : _ have pos is blk and class is O
word : รัฐมนตรีช่วยว่าการ have pos is ncn and class is PERSON
word : กระทรวง have pos is ncn and class is PERSON
word : สาธารณสุข have pos is ncn and class is PERSON
word : _ have pos is blk and class is O
word : แถลง have pos is vt and class is O
word : ที่ have pos is prel and class is O
word : จังหวัด have pos is ncn and class is O
word : เชียงใหม่ have pos is npn and class is LOCATION
word : ว่า have pos is conjncl and class is O
word : ผู้ have pos is pref2 and class is PERSON
word : ป่วย have pos is vi and class is PERSON
word :  have pos is None and class is O
word : ไข้หวัดนก have pos is vi and class is DISEASE
word : เกือบ have pos is adv and class is DISEASE
word : ร้อยทั้งร้อย have pos is idm and class is DISEASE
word : มี have pos is vt and class 